# Limpiando MetroNetwork 

In [1]:
include("graphs_metro.txt");

### `lineas_metro.txt`

Contiene las *variables* <br>
**`LineaN:`** N-ésima línea con el nombre de las estaciónes. <br>
**`Express_LíneaN:`** N-ésima línea Express (línea de sólo transbordos)<br>
**`Todas_Lineas:`** Arreglo donde cada entrada es una línea entera; así `Todas_Lineas[i][j]` nos dará la estación `j` de la línea `i`. <br>
**`Transfer_List:`** Arreglo de todos los transbordos de `Todas_Líneas`.

y las  *funciones*: <br>
**`Length_Lineas(total_lines):`** Número de estaciones de cada línea, en orden de línea. <br>
**`Length_Estaciones(total_lines):`** Número total de estaciones en toda la red <br>
**`GetStationNumber(station_name, line_number, total_lines) :`** Se da el nombre de una estación y la línea en donde se encuentra, y regresa el número de estación que le corresponde de la red. <br>
**`GetStationName(station_number, total_lines) :`** Dado el número de estación en la red, te regresa el nombre de estación que le corresponde. <br>

**Obs:** El default es siempre `Todas_Lineas`.

### `graphs_metro.txt`

Contiene las *funciones* <br>
**`MetroNetwork(total_lines):`** Hace una red tipo `Graph` con el número de estaciones totales en el arreglo de líneas. <br>
**`ConnectSimpleNetwork(total_lines):`** Une las estaciones de cada línea con aristas, sin considerar transbordos.
**`ConnectLines(total_lines):`** Une con aristas los transbordos de la red. <br>
**`ConnectFullNetwork_Weighted(normal_edge, transit_edge,total_lines):`** Crea la *red* de todas las líneas, dándole un peso `normal_edge`entre estaciones y `transit_edge` entre transbordos. <br>
**`MatrixDist(transfer_list, total_lines):`** Crea una matriz `D` de todas las distancias entre estaciones. *Ej: `D[i,j]` = distancia entre i-ésima y j-ésima estación.* 

**Obs_1:** Falta incluir el "peso" de cada arista, dependiendo de su tipo. <br>
**Obs_2:** El default es siempre `Todas_Lineas`.

<img src="Mexico_City_metro.png"> 